In [ ]:
import sqlalchemy as sa
from sqlalchemy import create_engine, text

import numpy as np
import pandas as pd
from datetime import datetime, timedelta,date

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

from data_logger import setup_logger
import module_query_update
import module_broker_tag_compare 
import warnings
import os
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

def init():  
    istcok_db=module_query_update.getConnection('istock')
    rating_db=create_engine('sqlite:///result/rating_db.db')
    end_date = pd.read_sql("SELECT MAX(td) FROM model_st ms ;",istcok_db)
    end_date=end_date.loc[0][0].strftime('%Y-%m-%d')
    start_date = pd.read_sql("select max(trade_date) from saas_rating;",rating_db)
    start_date=start_date.loc[0][0]
    sql_rating = f'''SELECT MIN(td_date) AS min_td_date
    FROM (
        SELECT  DISTINCT td_date 
        FROM shsz_stock_daily_quotation
        WHERE td_date <= '{end_date}'
        ORDER BY td_date DESC
        LIMIT 5
    ) AS recent_dates;'''
    date_df=pd.read_sql(sql_rating,istcok_db)
    from_date=date_df.loc[0][0].strftime('%Y-%m-%d')
    del date_df
    query_config= {'start_date': start_date,
                'end_date': end_date,
                'istock_conn': istcok_db,
                'rating_conn' : rating_db,
                'from_date': from_date}
    return query_config

In [ ]:
query_config=init()
example_sql=f'''SELECT stock_code,trade_date,wanlian_rating AS saas_rating_example FROM saas_rating WHERE trade_date = '{query_config['end_date']}';'''
topbrid_sql=f"""SELECT si.td_date AS trade_date,
si.stock_code,
si.stock_name AS sec_short_name_cn,
si.five_class_result_saas_adj_ori AS five_class_result_private_adj, 
ms.five_class_result_saas_reason 
FROM saas_index si 
LEFT JOIN model_st ms 
ON si.stock_code = ms.stock_code COLLATE utf8mb4_unicode_ci AND si.td_date = ms.td
WHERE si.td_date = '{query_config['end_date']}';"""
topbrid_df=pd.read_sql(topbrid_sql,query_config['istock_conn'])

example_df=pd.read_sql(example_sql,query_config['rating_conn'])

# example_df=module_broker_tag_compare.get_rating(query_config,query_config['end_date'],query_config['end_date'])[['trade_date','stock_code','saas_five_class_result_adj']]
# example_df.rename(columns={'saas_five_class_result_adj':'saas_rating_example'},inplace=True)


def generate_ths_code_and_td_mkt(stock_code):
    if stock_code.startswith(('60', '68')):
        return (f"{stock_code}.SH", "上交所")
    elif stock_code.startswith(('00', '30')):
        return (f"{stock_code}.SZ", "深交所")
    else:
        return (f"{stock_code}.BJ", "北交所")

# 应用函数并创建新列
example_df[['ths_code', 'td_mkt']] = example_df.apply(lambda row: generate_ths_code_and_td_mkt(row['stock_code']), axis=1, result_type='expand')

example_df['trade_date']=pd.to_datetime(example_df['trade_date'])
topbrid_df['trade_date']=pd.to_datetime(topbrid_df['trade_date'])
example_df.columns=['trade_date','sec_code', 'saas_rating_example', 'stock_code', 'td_mkt']
compare_test=pd.merge(example_df[['trade_date','stock_code','saas_rating_example']],
         topbrid_df[['stock_code','trade_date','sec_short_name_cn','five_class_result_private_adj','five_class_result_saas_reason']],
         on=['trade_date','stock_code'],
         how='left')
compare_test['is_mismatch'] = (compare_test['saas_rating_example'] != compare_test['five_class_result_private_adj']).astype(int)

In [6]:
def generate_ths_code_and_td_mkts(stock_code):
    if stock_code.startswith(('60', '68')):
        return f"{stock_code}.SH"
    elif stock_code.startswith(('00', '30')):
        return f"{stock_code}.SZ"
    else:
        return f"{stock_code}.BJ" 

stock_code='000720'
stock_codes=generate_ths_code_and_td_mkts(stock_code)
example_factor_query=f'''SELECT stock_code,trade_date,
five_class_result_adj AS five_class_result_saas,
listed_date,
rolling_avg_float_market_value_rank AS float_market_value_avg_rank,
rolling_avg_trans_amt_rank AS volume_avg_rank,
discount_rate,
collateral_ratio,
goodwill_ratio
FROM saas_indicator si 
WHERE trade_date='{query_config['end_date']}' AND stock_code='{stock_code}';'''

topbrid_factor_query=f'''SELECT stock_code, td_date AS trade_date,
five_class_result_saas,
listed_date,
float_market_value_avg_rank,
volume_avg_rank,
discount_rate,
collateral_ratio,
goodwill_ratio
FROM saas_index 
WHERE td_date='{query_config['end_date']}' AND stock_code = '{stock_codes}';'''

example_factor=pd.read_sql(example_factor_query,query_config['rating_conn'])
topbrid_factor=pd.read_sql(topbrid_factor_query,query_config['istock_conn'])

In [7]:
pd.concat([example_factor,topbrid_factor],axis=0)

,stock_code,trade_date,five_class_result_saas,listed_date,float_market_value_avg_rank,volume_avg_rank,discount_rate,collateral_ratio,goodwill_ratio
0,000720,2025-03-11,C,1997-05-09,0.533445,0.899574,0.0,5.32,0.0005
0,000720.SZ,2025-03-11,C,1997-05-09,0.532300,0.901200,0.0,5.00,0.0005
